In [ ]:
import sys
from datasets import load_dataset
import torch
from trl.trainer.dpo_trainer import DPOTrainer
import gc
import shutil
import os

sys.path.append('..')
from model.causal_lm import AutoLMFineTuner
from model.utils import LMHyperParams, SmModel, ModelChoice
from train import start_training
from dataset.squad import UltraFeedbackDataModule
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
params = LMHyperParams(
    base_model_checkpoint="Qwen/Qwen2.5-0.5B",
    warmup_steps_count=10,
    tuning_type="dpo",
    train_batch_size=1,
    gradient_accumulation_steps=1,
    optimizer="AdamW",
)

# https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/dpo-align-llms-in-2024-with-trl.ipynb

In [ ]:
model = AutoLMFineTuner(params)
model.to('cuda')

In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [8]:
data_module = UltraFeedbackDataModule(2, model.tokenizer, 1024, 10)
if os.path.exists(data_module.cache_dir):
    shutil.rmtree(data_module.cache_dir)
data_module.cpu_count = 12
data_module.setup("fit")

2024-11-23 17:00:54.615 | INFO     | dataset.squad:setup:219 - Loading dataset for stage fit
2024-11-23 17:00:55.809 | INFO     | dataset.squad:setup:230 - Processing dataset for stage fit, workers: 12, cache dir dataset_caches/ultrafeedback
num_proc must be <= 9. Reducing num_proc to 9 for dataset of size 9.
Parameter 'function'=<bound method UltraFeedbackDataModule.process_samples_batch of <dataset.squad.UltraFeedbackDataModule object at 0x1469a90766d0>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map (num_proc=9): 100%|██████████| 9/9 [00:02<00:00,  3.56 examples/s]
num_

In [11]:
data_module.precompute_reference_logprobs(model)

Precomputing reference logprobs:   0%|          | 0/5 [00:00<?, ?it/s]

Precomputing reference logprobs: 100%|██████████| 5/5 [00:21<00:00,  4.28s/it]
2024-11-23 17:16:15.292 | INFO     | dataset.squad:precompute_reference_logprobs:303 - Converting dataset


In [ ]:
for batch in data_module.train_dataloader():
    for k, v in batch.items():
        batch[k] = v.to('cuda')
    loss = model._step(batch)
    print(loss.item())
    break

In [ ]:
start_training(
    "16-mixed", "", model, model.tokenizer, data_module, params, False, ""
)
